As bibliotecas já estão instaladas na sala de aula. Se você estiver executando este notebook na sua própria máquina, pode instalar o seguinte:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

- Import libraries, API and LLM

In [1]:
from crewai import Agent, Task, Crew

In [11]:
support_agent = Agent(
    role="Representante Sênior de Suporte",
    goal="Ser o representante de suporte mais amigável e útil "
         "da sua equipe.",
    backstory=(
        "Você trabalha na crewAI (https://crewai.com) e agora "
        "está prestando suporte para {customer}, um cliente super importante "
        "para sua empresa."
        "Você precisa garantir que está oferecendo o melhor suporte possível!"
        "Certifique-se de fornecer respostas completas e detalhadas, "
        "sem fazer suposições."
    ),
    allow_delegation=False,
    verbose=True
)


> Ao não definir `allow_delegation=True`, `allow_delegation` assume seu valor padrão, que é True.

> Isso significa que o agente pode delegar seu trabalho a outro agente que seja mais adequado para realizar uma determinada tarefa.

In [3]:
support_quality_assurance_agent = Agent(
    role="Especialista em Garantia de Qualidade de Suporte",
    goal="Ser reconhecido por oferecer a melhor garantia de qualidade "
         "de suporte da sua equipe.",
    backstory=(
        "Você trabalha na crewAI (https://crewai.com) e agora "
        "está colaborando com sua equipe "
        "em uma solicitação de {customer}, garantindo que o "
        "representante de suporte esteja "
        "oferecendo o melhor suporte possível.\n"
        "Você precisa assegurar que o representante de suporte esteja fornecendo "
        "respostas completas e detalhadas, sem fazer suposições."
    ),
    allow_delegation=True,
    verbose=True,
)

>**Interpretação de Papéis (Role Playing):** Ambos os agentes receberam um papel, objetivo e história de fundo.

>**Foco:** Ambos os agentes foram orientados a se aprofundar no personagem dos papéis que estão desempenhando.

>**Colaboração:** O Agente de Garantia de Qualidade de Suporte pode delegar o trabalho de volta ao Agente de Suporte, permitindo que esses agentes trabalhem juntos de forma colaborativa.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [4]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Ferramentas Personalizadas Possíveis
- Carregar dados do cliente
- Acessar conversas anteriores
- Carregar dados de um CRM
- Verificar relatórios de bugs existentes
- Verificar solicitações de funcionalidades existentes
- Verificar tickets em andamento
- ... e mais

- Algumas maneiras de utilizar as tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instanciar uma ferramenta de raspagem de documentos.  
- A ferramenta irá raspar uma página (apenas 1 URL) da documentação do CrewAI.

In [5]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Diferentes Formas de Fornecer Ferramentas aos Agentes

- **Nível do Agente**: O Agente pode usar a(s) Ferramenta(s) em qualquer Tarefa que executar.  
- **Nível da Tarefa**: O Agente usará a(s) Ferramenta(s) somente ao executar aquela Tarefa específica.  

**Nota**: Ferramentas de Tarefa substituem as Ferramentas do Agente.

### Criando as tasks


In [6]:
inquiry_resolution = Task(
    description=(
        "{customer} acabou de entrar em contato com uma solicitação super importante:\n"
        "{inquiry}\n\n"
        "{person} de {customer} foi quem entrou em contato. "
        "Certifique-se de usar tudo o que você sabe "
        "para oferecer o melhor suporte possível."
        "Você deve se esforçar para fornecer uma resposta "
        "completa e precisa à solicitação do cliente."
    ),
    expected_output=(
        "Uma resposta detalhada e informativa à "
        "solicitação do cliente, que aborde "
        "todos os aspectos da pergunta.\n"
        "A resposta deve incluir referências "
        "a tudo que você utilizou para encontrar a solução, "
        "incluindo dados externos ou soluções. "
        "Certifique-se de que a resposta seja completa, "
        "sem deixar perguntas sem resposta, e mantenha um tom "
        "amigável e útil ao longo de toda a interação."
    ),
    tools=[docs_scrape_tool],
    agent=support_agent,
)

>`quality_assurance_review` não está utilizando nenhuma ferramenta.

>Neste caso, o Agente de Garantia de Qualidade (QA) revisará apenas o trabalho do Agente de Suporte.

In [7]:
quality_assurance_review = Task(
    description=(
        "Revisar a resposta elaborada pelo Representante Sênior de Suporte para a solicitação de {customer}. "
        "Certifique-se de que a resposta seja abrangente, precisa e atenda aos "
        "padrões de alta qualidade esperados para o suporte ao cliente.\n"
        "Verifique se todas as partes da solicitação do cliente "
        "foram abordadas "
        "de forma completa, com um tom útil e amigável.\n"
        "Confira as referências e fontes utilizadas para "
        "encontrar as informações, "
        "garantindo que a resposta esteja bem fundamentada e "
        "não deixe perguntas sem resposta."
    ),
    expected_output=(
        "Uma resposta final, detalhada e informativa, "
        "pronta para ser enviada ao cliente.\n"
        "Esta resposta deve abordar completamente a "
        "solicitação do cliente, incorporando todo o "
        "feedback e melhorias relevantes.\n"
        "Não seja muito formal, somos uma empresa descontraída e descolada, "
        "mas mantenha um tom profissional e amigável em toda a interação."
    ),
    agent=support_quality_assurance_agent,
)


### Criando a Equipe

#### Memória
- Definir `memory=True` ao montar a equipe ativa a Memória.

In [8]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True
)

### Executando a Crew

#### Restrições  
- Ao executar o código abaixo, você pode observar que os agentes e as respostas estão dentro do escopo esperado para eles.

In [12]:
inputs = {
    "customer": "NeroAI",
    "person": "Enzo",
    "inquiry": "Preciso de ajuda para configurar uma Crew "
               "e iniciá-la, especificamente "
               "como posso adicionar memória à minha Crew? "
               "Você pode fornecer orientação?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Representante Sênior de Suporte
## Task: NeroAI acabou de entrar em contato com uma solicitação super importante:
Preciso de ajuda para configurar uma Crew e iniciá-la, especificamente como posso adicionar memória à minha Crew? Você pode fornecer orientação?

Enzo de NeroAI foi quem entrou em contato. Certifique-se de usar tudo o que você sabe para oferecer o melhor suporte possível.Você deve se esforçar para fornecer uma resposta completa e precisa à solicitação do cliente.


# Agent: Representante Sênior de Suporte
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 
Introduction - CrewAI CrewAI home page Search CrewAI docs Search... Navigation Get Started Introduction Get Started Examples CrewAI home page Community Changelog Get Started Introduction Installation Quickstart Core Concepts Agents Tasks Crews Flows Knowledge LLMs Processes Collaboration Training Memory Planning Testing CLI Tools Using LangChain Tools Using LlamaIndex Tools How to Guides Cre

- Resultado em Markdown.

In [13]:
from IPython.display import Markdown
Markdown(result.raw)

Olá Enzo! Fico feliz em ajudar com a configuração da sua Crew na plataforma crewAI e na adição de memória. Vou te guiar pelo processo, passo a passo!

### 1. Criar uma Crew
Para criar sua Crew, siga os passos abaixo:
- Acesse o [painel de controle da crewAI](https://crewai.com).
- Vá até a seção de gerenciamento de Crews.
- Clique em **"Criar Nova Crew"**.
- Preencha as informações necessárias como nome e descrição, refletindo as atividades e objetivos da Crew.

### 2. Adicionar Memória à Crew
Uma vez que sua Crew esteja criada, você pode adicionar memória. A memória é fundamental para armazenar informações e contextos relevantes para suas interações. Para adicionar memória, siga estas etapas:
- **Configuração do Módulo de Memória**: Você precisará criar ou integrar um módulo de memória que armazene interações anteriores e aprendizados, que os agentes da Crew podem consultar em futuras interações.
- Você pode usar uma estrutura de dados como banco de dados ou um sistema de gerenciamento de memória que atenda às suas necessidades específicas.

### 3. Configuração de Agentes
Dentro da sua Crew, você terá a possibilidade de configurar agentes com funções específicas, como:
- **Analisadores de dados**: Responsáveis por interpretar e analisar informações passadas.
- **Tomadores de decisão**: Agentes que utilizarão a memória para fundamentar suas decisões.

Certifique-se de que cada agente tenha acesso ao módulo de memória que foi configurado, para permitir um uso eficaz das informações armazenadas.

### 4. Documentação
Para orientações adicionais e práticas detalhadas sobre a implementação de memória em suas Crews, recomendo consultar nossa documentação oficial sobre como criar e dar início a uma Crew, que pode ser encontrada em [Documentação crewAI](https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/). 

### 5. Testando a Configuração
Depois de adicionar a memória e configurar seus agentes, é essencial testar:
- Realize testes para garantir que os dados estão sendo salvos e acessados corretamente. Esses testes são cruciais para identificar potenciais problemas antes que a Crew comece suas operações.

### Conclusão
Caso você tenha mais dúvidas ou precise de assistência em qualquer uma das etapas, não hesite em entrar em contato! Estou aqui para garantir que sua Crew funcione perfeitamente e atenda a todas as suas expectativas. Vamos juntos fazer sua Crew brilhar! 🚀✨